In [1]:
import pandas as pd 
from collections import defaultdict

In [2]:
tlkt_df = pd.read_csv("DATABASE.xlsx - TLKT.csv")


In [3]:
print (tlkt_df.head())

           Kỳ thi        Ngày Loại tư liệu kỳ thi Tên file (đích)  \
0  Free Contest 1  03/12/2015  Lời giải (thư mục)        Lời giải   
1  Free Contest 1  03/12/2015    Đề bài (thư mục)          Đề bài   
2  Free Contest 1  03/12/2015              Đề bài       ASCII.pdf   
3  Free Contest 1  03/12/2015              Đề bài      KDIGIT.pdf   
4  Free Contest 1  03/12/2015              Đề bài    KEYBOARD.pdf   

                                   Link Google Drive  
0  https://drive.google.com/drive/folders/1jg7mtI...  
1  https://drive.google.com/drive/folders/1hZigcQ...  
2  https://drive.google.com/file/d/1jc3vQcj5tCO3t...  
3  https://drive.google.com/file/d/1kPQNe4kZ7_I4D...  
4  https://drive.google.com/file/d/1tHdavcOnQLDyi...  


In [12]:
from googleapiclient.discovery import build
from googleapiclient.http import MediaIoBaseDownload
from googleapiclient.errors import HttpError
import google.auth
import io, json
from pathlib import Path

class GoogleDriveFile: # refer to the "gfile" attribute 

    def __init__(self, name = "", link = "") -> None:
        self.name = name 
        self.link = link
        self.extension = name[-3:]
        _u = (link.split('/'))
        self.id = "" 
        for u in _u: 
            if len(u) > len(self.id):
                self.id=u
        pass

    def __str__(self) -> str:
        return f"File name: {self.name}, file id {self.id}."

    def isempty(self) -> bool: 
        return not bool(self.link)
    
    def download(self, destination):
        #put json credentials her from service account or the like
        # More info: https://cloud.google.com/docs/authentication

        credz, _ = google.auth.load_credentials_from_file("gdrivefile-downloader-ef880c6487fc.json")
        drive_service = build('drive', 'v3', credentials=credz)
        request = drive_service.files().get_media(fileId=self.id)
        fh = io.FileIO(destination, 'wb') # this can be used to write to disk
        downloader = MediaIoBaseDownload(fh, request)
        done = False
        while done is False:
            try:
                status, done = downloader.next_chunk()
                print("Download %d%%." % int(status.progress() * 100))
            except HttpError as error:
                print(f'[{self.id}][{destination}] An error occurred: {error}')
                break
           


class Submission:
    def __init__(self) -> None:
        self.gfile = GoogleDriveFile() 
        self.status = "main" 
        # status in ["main", "correct", "incorrect"]
        # note: this is a simplified version of Polygon status system  
        pass

class Task: 
    def __init__(self) -> None:
        self.name = "" 
        self.statement_gfile = GoogleDriveFile() 
        self.submissions = [] # list of Submissions associated with task, typically solutions   
        self.editorial_gfile = GoogleDriveFile() 
        self.testdata_gfile = GoogleDriveFile() 
        pass

    def __str__(self) -> str:
        ret = f" Task name: {self.name}\n"
        return ret

class Contest:
    def __init__(self) -> None:
        self.name = ""
        self.tasks = defaultdict(Task)
        self.ranking_gfile = GoogleDriveFile()
        pass

    def __str__(self) -> str:
        ret = f"Contest name: {self.name}\nContest Details:\nRanking file: {self.ranking_gfile}\n"
        for task in self.tasks: 
            ret += self.tasks[task].__str__()
        return ret

In [13]:
contests = defaultdict(Contest) 

for row_id in range(0, len(tlkt_df)): 
    contest = tlkt_df["Kỳ thi"][row_id]
    contests[contest].name = contest
    tlkt_type = tlkt_df["Loại tư liệu kỳ thi"][row_id]
    file_name = tlkt_df["Tên file (đích)"][row_id]
    task = file_name[:-4]
    file_glink = tlkt_df["Link Google Drive"][row_id] 
    file = GoogleDriveFile(file_name, file_glink)

    if tlkt_type == "Đề bài":
        contests[contest].tasks[task].statement_gfile = file
        contests[contest].tasks[task].name = task
    elif tlkt_type == "Bảng điểm": 
        if (file.extension == "png"):
            if contests[contest].ranking_gfile.name == "":
                contests[contest].ranking_gfile = file    
        if (file.extension == "csv") or (file.extension == "txt"):
            contests[contest].ranking_gfile = file
    

In [14]:
ranking_ext = defaultdict(int)
for contest in contests:
    # print(contest[1])
    ranking_ext[contests[contest].ranking_gfile.extension] += 1
    # print(contests[contest].ranking_gfile) 
print (len(contests.items())) 
print (ranking_ext)

183
defaultdict(<class 'int'>, {'txt': 88, 'png': 34, '': 55, 'csv': 6})


In [8]:
localdir_head = "TLKT/"
for contest_name, contest in contests.items():
    if (not contest.ranking_gfile.isempty()):
        localdir = localdir_head + contest_name + "/" 
        Path(localdir).mkdir(parents=True, exist_ok=True)
        localdir += "ranking." + contest.ranking_gfile.extension;
        contest.ranking_gfile.download(localdir)

Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
[0B97Iu255Omo8aDhQSnl3MkFUUkU][TLKT/Free Contest 35/ranking.txt] An error occurred: <HttpError 404 when requesting https://www.googleapis.com/drive/v3/files/0B97Iu255Omo8aDhQSnl3MkFUUkU?alt=media returned "File not found: 0B97Iu255Omo8aDhQSnl3MkFUUkU.". Details: "[{'domain': 'global', 'reason': 'notFound', 'message': 'File not found: 0B97Iu255Omo8aDhQSnl3MkFUUkU.', 'locationType': 'parameter', 'location': 'fileId'}]">
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
Download 100%.
[0B38ZD6VWm0VJTFg4UDhCLUNwcUk][TLKT/Testing Round 2.1/ranking.txt] An error occurred: <HttpError 4